## Imports

In [1]:
!pip install numpy pyscipopt

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.13/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [ ]:
import numpy as np
from pyscipopt import Model, quicksum

In [ ]:
N = 10
K = 18

I = np.arange(N)
I_s = [0,10]
J = np.arange(K)

In [ ]:
from enum import Enum

class SymetricScheme(Enum):
	MIRRORED = 0
	FRENCH = 1
	ENGLISH = 2
	INVERTED = 3
	B2B = 4

In [ ]:
class FootballScheduler:
    def __init__(self, N: int, K:int, I_s: list[int], scheme: SymetricScheme):
        if(max(I_s)) > N:
            raise ValueError("I_s must be a subset of teams I")
        self.N = N
        self.K = K
        self.I_s = I_s
        self.scheme = scheme
        self.x = {}
        self.y = {}
        self.w = {}
        self.model = Model("Football Scheduler")

    def __instance_vars(self):
        '''
		Define the decision variables
		Variables:
			x[i,j,k] = 1 if team i plays against team j in round k
			y[i,k] = 1 if team i plays at home in round k
			w[i,k] = 1 if team i plays away in round k
		'''
        for i in range(N):
            for j in range(N):
                for k in range(K):
                    self.x[i,j,k] = self.model.addVar(vtype="B", name=f"x_{i}_{j}_{k}")

        for i in range(N):
            for k in range(K):
                self.y[i, k] = self.model.addVar(vtype="B", name=f"y_{i}_{k}")
                self.w[i, k] = self.model.addVar(vtype="B", name=f"w_{i}_{k}")

    def __instance_base_constraints(self):
        """
        Define the base constraints
        Constraints:
			- Double round robin constraints
			- Compactness constraints
			- Top-teams constraints
        """
        # Double round robin constraints.
        for i in range(self.N):
            for j in range(self.K):
                if i == j:
                    continue
                # C1- every team faces every other team once in the first half
                self.model.addCons(
                    quicksum(self.x[i, j, k] + self.x[j, i, k] for k in range(self.N))
                    == 1,
                    name=f"match_first_half_{i}_{j}",
                )
                # C2 - every team faces every other team once in the second half
                self.model.addCons(
					quicksum(self.x[i, j, k] + self.x[j, i, k] for k in range(self.N, self.K, 1)) == 1,
					name=f"match_second_half_{i}_{j}",
				)
                # C3 - exactly one of the two games is played at home while the other one is played away
                self.model.addCons(
					quicksum(self.x[i, j, k] for k in range(self.K)) == 1,
					name=f"match_second_half_{i}_{j}",
				)

        # Compactness constraints
        for j in range(self.N):
            for k in range(self.K):
                # C4 - all teams must play one match in each round.
                self.model.addCons(
					quicksum(self.x[i, j, k] + self.x[j, i, k] for i in range(self.N) if i != j) == 1,
					name=f"one_match_per_round_{j}_{k}",
				)

        # Top-teams constraints
        for i in [x for x in range(N) if x not in I_s]:
            for k in range(K):
                for j in I_s:
                    # C5 - No non-top team be required to play against any of the top teams in consecutive matches.
                    self.model.addCons(
						quicksum(self.x[i, j, k] + self.x[j, i, k] + self.x[i, j, k + 1] + self.x[j, i, k + 1])
						<= 1,
						name=f"top_team_cons_{i}_{j}_{k}",
					)

In [ ]:
M = Model("Football Scheduler")

x = {}
y = {}
w = {}

for i in range(N):
	for j in range(N):
		for k in range(K):
			x[i,j,k] = M.addVar(vtype="B", name=f"x_{i}_{j}_{k}")

for i in range(N):
	for k in range(K):
		y[i, k] = M.addVar(vtype="B", name=f"y_{i}_{k}")
		w[i, k] = M.addVar(vtype="B", name=f"w_{i}_{k}")

In [ ]:
# Double round robin constraints.
for i in range(N):
    for j in range(K):
        if i == j:
            continue
        # C1- every team faces every other team once in the first half
        M.addCons(
            quicksum(x[i, j, k] + x[j, i, k] for k in range(N)) == 1,
            name=f"match_first_half_{i}_{j}",
        )
        # C2 - every team faces every other team once in the second half
        M.addCons(
            quicksum(x[i, j, k] + x[j, i, k] for k in range(N, K, 1)) == 1,
            name=f"match_second_half_{i}_{j}",
        )
        # C3 - exactly one of the two games is played at home while the other one is played away
        M.addCons(
            quicksum(x[i, j, k] for k in range(K)) == 1,
            name=f"match_second_half_{i}_{j}",
        )

# Compactness constraints
for j in range(N):
    for k in range(K):
        # C4 - all teams must play one match in each round.
        M.addCons(
			quicksum(x[i, j, k] + x[j, i, k] for i in range(N) if i != j) == 1,
			name=f"one_match_per_round_{j}_{k}",
		)

# Top-teams constraints
for i in [x for x in I if x not in I_s]:
	for k in range(K):
		for j in I_s:
			# C5 - No non-top team be required to play against any of the top teams in consecutive matches.
			M.addCons(
				quicksum(x[i, j, k] + x[j, i, k] + x[i, j, k + 1] + x[j, i, k + 1])
				<= 1,
				name=f"top_team_cons_{i}_{j}_{k}",
			)

_IncompleteInputError: incomplete input (598713993.py, line 23)